# Extração de características em imagens 



In [1]:
import pickle
import numpy as np
import cv2
from scipy.stats import skew

## Conjunto de dados

O conjunto de dados utilizado é o CIFAR-10. O CIFAR-10 é um dataset desenvolvido pelo instituto canadense de pesquisa avançada (CIFAR) a partir de 10 classes de imagens, cada uma com 6000 imagens de 32x32 pixels.

Ao baixar os dados utilizando os comandos de ```rotina.sh```, eles vem em batches, que são arquivos binários formados por arrays com os valores RGB de cada imagem. Os arquivos data_batch possuem os dados que serão usados para o treino, sendo cada um formado de 10000 arrays de 3072 linhas, com 1024 entradas de valores de Vermelho (R), Azul (B) e Verde (G) formando os pixels da imagem 32x32.

Vamos transformar esses arrays de 3072 linhas em arrays de 1024 linhas de três entradas cada



In [2]:
def load_batch():
    path = 'cifar-10-batches-py/'

    imagens = []
    labels = np.zeros(50000)
    count = 0

    for i in range(1, 6, 1):
        file = f'data_batch_{i}'
        print(file)
        f = open(path+file, 'rb')
        df = pickle.load(f, encoding='latin1')

        images = df['data']
        #images = np.reshape(images, (10000, 3, 32, 32))
        label = df['labels']
        imagearray = np.array(images)   #   (10000, 3072)
        labelarray = np.array(label)   #   (10000,)

        for j in range(0, len(imagearray), 10):
            image = np.transpose(np.reshape(imagearray[j], (3, 32, 32)), (1, 2, 0))
            imagens.append(image)
            labels[count+j] = labelarray[j]

        count = count + 10000

    return imagens, labelarray

def load_test_batch():
    path = 'cifar-10-batches-py/'
    file = 'test_batch'

    f = open(path+file, 'rb')
    dict = pickle.load(f, encoding='latin1')
    images = dict['data']
    #images = np.reshape(images, (10000, 3, 32, 32))
    labels = dict['labels']
    imagearray = np.array(images)   #   (10000, 3072)
    labelarray = np.array(labels)   #   (10000,)

    imagens = []
    label = []

    for j in range(0, len(imagearray), 10):
        image = np.transpose(np.reshape(imagearray[j], (3, 32, 32)), (1, 2, 0))
        imagens.append([image])
        label.append(labelarray[j])
    
    return imagearray, labelarray

imagearray, labelarray = load_batch()
test_image, test_label = load_test_batch()
print(len(imagearray))
print(len(test_image))


data_batch_1
data_batch_2
data_batch_3
data_batch_4
data_batch_5
5000
10000


## Extração de características

### Características de cor

Como as imagens estão usando o espaço de cor BGR, vamos utilizar características de cor relacionadas ao espaço RGB, como por exemplo:

* Momentos de Cor (CM): Utiliza cálculos de média, desvio padrão e assimetria da imagem, cálculados para cada cor.

In [3]:
def CM(imagens):
    medias = []
    stds= []
    skews = []
    for imagem in imagens:

        azul = imagem[:, :, 0]
        verde = imagem[:, :, 1]
        vermelho = imagem[:, :, 2]
        
        # Calculo de médias
        media_azul = np.mean(azul)
        media_verde = np.mean(verde)
        media_vermelho = np.mean(vermelho)
        media = [media_azul, media_verde, media_vermelho]
        medias.append(media)

        # Calculo de desvio padrão
        std_azul = np.std(azul)
        std_verde = np.std(verde)
        std_vermelho = np.std(vermelho)
        std = [std_azul, std_verde, std_vermelho]
        stds.append(std)

        # Calculo da assimetria
        skew_azul = skew(azul)
        skew_verde = skew(verde)
        skew_vermelho = skew(vermelho)
        assim = [skew_azul, skew_verde, skew_vermelho]
        # print(assim)
        skews.append(assim)
        

CM(imagearray)

/tmp/ipykernel_9636/244653972.py:26: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew_azul = skew(azul)
/tmp/ipykernel_9636/244653972.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew_verde = skew(verde)
/tmp/ipykernel_9636/244653972.py:28: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew_vermelho = skew(vermelho)


### Características de textura

Como extrator de características de textura, vamos utilizar o Filtro de Gabor. O filtro de gabor faz a amostragem do domínio da frequência inteiro de uma imagem pelas características de frequência central e parâmetros de orientação.

In [5]:
def gabor_filter(imagens):
    sigma = 8.0
    theta = np.pi / 4
    lambd = 10.0
    gamma = 0.5
    psi = 0
    gabor_kernel = cv2.getGaborKernel((21, 21), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)

    # for img in imagens:
    filtered_imgs = []
    for img in imagens:
        filtered_imgs.append(
            cv2.filter2D(
                cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
                cv2.CV_8UC3,
                gabor_kernel
            )
        )

    return filtered_imgs

filtered_imgs = gabor_filter(imagearray)

img = cv2.cvtColor(np.concatenate(imagearray[0:5], axis=1), cv2.COLOR_BGR2GRAY)
filtered_img = np.concatenate(filtered_imgs[0:5], axis = 1)

cv2.imshow("gabor", np.concatenate((img, filtered_img), axis=0))
# cv2.imshow('imagem nao alterada', img)
# cv2.imshow('teste', filtered_img)
cv2.waitKey(0)
cv2.destroyAllWindows()



## Análise utilizando Machine Learning

O algoritmo de machine learning utilizado para classificar as imagens será o Support Vector Machine



## Bibliografia
https://gvpress.com/journals/IJMUE/vol8_no4/39.pdf
Kumar, G., & Bhatia, P. K. (2014). A Detailed Review of Feature Extraction in Image Processing Systems. 2014 Fourth International Conference on Advanced Computing & Communication Technologies. doi:10.1109/acct.2014.74
Wang, X., Ding, X., & Liu, C. (2005). Gabor filters-based feature extraction for character recognition. Pattern Recognition, 38(3), 369–379. doi:10.1016/j.patcog.2004.08.00